In [52]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras import backend as K
K.set_image_data_format('channels_last')
import numpy

In [53]:
seed = 5
numpy.random.seed(seed)

In [54]:
'''
this part is required to solve
AttributeError: module 'tensorflow_core._api.v2.config' has no attribute 'experimental_list_devices'.
This error occurs because the experimental_list_devices is deprecated in tf 2.1.
'''
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback
print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)
def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus

tf.__version__ is 2.1.0
tf.keras.__version__ is: 2.2.4-tf


In [55]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')
x_train = x_train/255
x_test = x_test/255
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [56]:
def larger_conv():
    model = Sequential()
    model.add(Conv2D(30, (5,5), input_shape=(28,28,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(15, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [57]:
model = larger_conv()
datagen = ImageDataGenerator(featurewise_center=True, zca_whitening=True)
datagen.fit(x_train)
it = datagen.flow(x_train, y_train, batch_size=200)
model.fit_generator(it, validation_data=(x_test, y_test), epochs=5, steps_per_epoch=300,
                    verbose=1)
scores = model.evaluate(x_test, y_test, verbose=0)

Epoch 1/5
300/300 [==============================] - 31s 102ms/step - loss: 0.5034 - accuracy: 0.8450 - val_loss: 0.4807 - val_accuracy: 0.8244
Epoch 2/5
300/300 [==============================] - 33s 108ms/step - loss: 0.1144 - accuracy: 0.9651 - val_loss: 0.3796 - val_accuracy: 0.8646
Epoch 3/5
300/300 [==============================] - 35s 115ms/step - loss: 0.0804 - accuracy: 0.9747 - val_loss: 0.3238 - val_accuracy: 0.8913
Epoch 4/5
300/300 [==============================] - 35s 118ms/step - loss: 0.0638 - accuracy: 0.9800 - val_loss: 0.2925 - val_accuracy: 0.8996
Epoch 5/5
300/300 [==============================] - 34s 112ms/step - loss: 0.0538 - accuracy: 0.9830 - val_loss: 0.2099 - val_accuracy: 0.9317


In [58]:
print("Baseline CNN: %.2f%%" %(scores[1]*100))

Baseline CNN: 93.17%
